In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import scipy.sparse as sp
import sys
import os
from utils import *
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_visible_devices(devices=gpus[0:3], device_type='GPU')

This layer computes for each node \(i\):
$$
    \Z_i = \textrm{MLP}\big( (1 + \epsilon) \cdot \X_i + \sum\limits_{j \in \mathcal{N}(i)} \X_j \big)
$$
where MLP is a multi-layer perceptron.

In [2]:
def convert_csr_to_SparseTensor(csr_matrix):
    if sp.isspmatrix_csr(csr_matrix):
            csr_matrix = csr_matrix.tocoo()
    row = csr_matrix.row
    col = csr_matrix.col
    pos = np.c_[row,col]
    data = csr_matrix.data
    shape = csr_matrix.shape
    return tf.SparseTensor(pos, data, shape)

In [3]:
dataset_name = "cora"

adj, whole_features, whole_labels, train_idx, val_idx, test_idx = load_data(dataset_name)
adj_SparseTensor = convert_csr_to_SparseTensor(adj)
features = preprocess_features(whole_features).todense()

num_nodes = features.shape[0]
feature_dim = features.shape[1]
epochs = 500
n_classes = whole_labels.shape[1]
dropout_rate = 0.6 
learning_rate = 5e-3
L2_reg = 5e-4

In [4]:
class GINLayer(keras.layers.Layer):
    def __init__(self, 
                 output_dim, 
                 mlp_num, 
                 hidden_units,
                 adj, 
                 num_nodes,
                 epsilon=0,
                 epsilon_learnable = True,
                 mlp_activation = keras.activations.relu,
                 activation = None,
                 use_bias = False):
        super(GINLayer,self).__init__(self)
        self.output_dim = output_dim,
        self.adj = adj
        self.epsilon = epsilon
        self.mlp_num = mlp_num
        self.mlp_activation = mlp_activation
        self.activation = activation
        self.use_bias = use_bias
        self.hidden_units = hidden_units  # 每个MLP的units数
        self.epsilon_learnable = epsilon_learnable
        self.num_nodes = num_nodes

    def build(self, input_shape):
        if self.hidden_units is not None and self.mlp_num > 0:
            self.mlp = keras.models.Sequential([
                keras.layers.Dense(units=units, activation=self.mlp_activation, use_bias=self.use_bias)
                for units in self.hidden_units
            ])
        self.output_dense = keras.layers.Dense(units=self.output_dim[0], use_bias=self.use_bias, activation= self.activation)
        if self.epsilon_learnable:
            self.eps = self.add_weight(name = "epsilon",
                            shape = (1,),
                            initializer=keras.initializers.zeros,
                            trainable=True)
        else:
            self.eps = tf.constant(self.epsilon)
        self.built = True
    



    def _message_passing(self, adj, features):
        index_i = adj.indices[:,0]
        index_j = adj.indices[:,1]
        messages = tf.gather(features, index_j)
        agg = tf.math.unsorted_segment_sum(messages, index_i, self.num_nodes)
        # agg = scatter_sum(messages, index_i, self.num_nodes)
        return agg

    def call(self, inputs):
        features = inputs
        assert keras.backend.is_sparse(self.adj), "Adjcency Matrix Must be Sparse Tensor"
        assert keras.backend.ndim(features), "Features Must be Rank 2"
        assert self.mlp_num == len(self.hidden_units) or self.hidden_units is None, "Number of MLP layers must be equal to hidden_units list"
        agg = self._message_passing(self.adj, features)
        if self.hidden_units is not None and self.mlp_num > 0:
            mlp_output = self.mlp((1.+ self.eps) * features + agg)
            out = self.output_dense(mlp_output)
        else:
            out = self.output_dense((1.+ self.eps) * features + agg)
        
        return out


In [5]:
model_input = keras.Input(shape = (feature_dim,))
# 第一层的MLP有两层
gin1 = GINLayer(output_dim=128, 
                num_nodes = num_nodes,
                mlp_num=1, 
                hidden_units=[128], 
                adj = adj_SparseTensor, 
                epsilon_learnable=True, 
                activation=keras.activations.relu)(model_input)
gin2 = GINLayer(output_dim=n_classes,
                num_nodes = num_nodes,
                mlp_num=1, 
                hidden_units=[16], 
                adj=adj_SparseTensor,
                epsilon_learnable=True, 
                activation=keras.activations.softmax)(gin1)
# gin1 = GINLayer(output_dim=128, 
#                 mlp_num=0, 
#                 hidden_units=None, 
#                 adj = adj_SparseTensor, 
#                 epsilon_learnable=True, 
#                 activation=keras.activations.relu)(model_input)

model = keras.models.Model(inputs = model_input, outputs = gin2)
model.compile(loss = keras.losses.categorical_crossentropy, optimizer = keras.optimizers.Adam(learning_rate=learning_rate),
              weighted_metrics=['categorical_crossentropy', 'acc'])
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1433)]            0         
_________________________________________________________________
gin_layer (GINLayer)         (2708, 128)               199809    
_________________________________________________________________
gin_layer_1 (GINLayer)       (2708, 7)                 2161      
Total params: 201,970
Trainable params: 201,970
Non-trainable params: 0
_________________________________________________________________


In [6]:
y_train, train_mask = generate_mask_data(whole_labels, train_idx)
y_val, val_mask = generate_mask_data(whole_labels, val_idx)
y_test, test_mask = generate_mask_data(whole_labels, test_idx)



history = model.fit(x =  features, 
                         y = y_train, 
                         sample_weight= train_mask,
                         validation_data=(features, y_val, val_mask),
                         batch_size = num_nodes, epochs = 300, 
                         shuffle = False, 
                         workers=10, 
                         use_multiprocessing=True)

c: 1.0000 - val_loss: 0.7048 - val_categorical_crossentropy: 3.8169 - val_acc: 0.7320
Epoch 208/300
1/1 [==============================] - 0s 67ms/step - loss: 8.7549e-06 - categorical_crossentropy: 1.6934e-04 - acc: 1.0000 - val_loss: 0.7082 - val_categorical_crossentropy: 3.8355 - val_acc: 0.7320
Epoch 209/300
1/1 [==============================] - 0s 75ms/step - loss: 8.5338e-06 - categorical_crossentropy: 1.6507e-04 - acc: 1.0000 - val_loss: 0.7117 - val_categorical_crossentropy: 3.8547 - val_acc: 0.7320
Epoch 210/300
1/1 [==============================] - 0s 65ms/step - loss: 8.3186e-06 - categorical_crossentropy: 1.6091e-04 - acc: 1.0000 - val_loss: 0.7153 - val_categorical_crossentropy: 3.8741 - val_acc: 0.7320
Epoch 211/300
1/1 [==============================] - 0s 86ms/step - loss: 8.1100e-06 - categorical_crossentropy: 1.5687e-04 - acc: 1.0000 - val_loss: 0.7189 - val_categorical_crossentropy: 3.8937 - val_acc: 0.7320
Epoch 212/300
1/1 [==============================] - 0s 75

In [8]:
eval_results = model.evaluate(features, y_test, sample_weight=test_mask, batch_size=num_nodes)

1/1 [==============================] - 0s 4ms/step - loss: 1.5001 - categorical_crossentropy: 4.0622 - acc: 0.7390
